In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

import paper_utils as pu

pu.set_plots()

# Load data

In [ ]:
data = {project: pu.load(project) for project in pu.PROJECTS}

# Figure 2

In [ ]:
def make_plot(data, cooling_load_unit, fig_name):
    models = {}
    f, ax = plt.subplots(2, 3, figsize=(pu.PAGE_WIDTH, pu.ROW_HEIGHT * 2), sharex=True)
    ax = ax.flatten()
    for i, project in enumerate(pu.PROJECTS):

        weekend = pu.choose_weekend[project]
        pu.plot_base(
            data[project],
            fax=(f, ax[i]),
            weekend=weekend,
            cooling_load_unit=cooling_load_unit,
        )
        models[project] = pu.create_model(
            data[project], fax=(f, ax[i]), weekend=weekend
        )
        ax[i].set_title(f"{pu.plot_numbers[i]}) {project}")
    for a in ax:
        a.xaxis.set_tick_params(labelbottom=True)
    f.tight_layout()
    pu.add_legend((f, ax))

    if pu.save_fig:
        f.savefig(fig_name)
    return models

In [ ]:
base_cooling_models = make_plot(data, "MJ_per_m2", pu.fig_path / f"Figure 2 A-F.pdf")

In [ ]:
f, ax = plt.subplots(1, 2, figsize=(pu.PAGE_WIDTH, 0.75 * pu.ROW_HEIGHT))
ax = ax.flatten()
for i, project in enumerate(pu.PROJECTS):
    print(project)
    pu.ci_plot(
        ax[0], i, base_cooling_models[project], "Treated", color=pu.dark_grey, verb=True
    )
    pu.ci_plot(
        ax[1], i, base_cooling_models[project], "Tmean", color=pu.dark_grey, verb=True
    )
    print()

for a in ax:
    a.set_xticks(range(len(pu.PROJECTS)))
    a.set_xticklabels(pu.PROJECTS, rotation=25, ha="right")
ax[0].set_ylabel("Impact (%)")
ax[1].set_ylabel("Impact (%/$\degree$F)")
ax[0].set_ylim(top=0)
ax[1].set_ylim(bottom=0)

title0 = (
    f"{pu.plot_numbers[6]}) "
    + "Impact of 2$\degree$F set point increase on energy for cooling"
)
title1 = f"{pu.plot_numbers[7]}) " + "Impact of mean daily OAT on energy for cooling"
ax[0].set_title(title0, ha="left", loc="left")
ax[1].set_title(title1, ha="left", loc="left")

f.tight_layout(w_pad=2.0)
plt.subplots_adjust(top=0.85, bottom=0.27)
if pu.save_fig:
    f.savefig(pu.fig_path / "Figure 2 G-H.pdf")

In [ ]:
def y_transform(y):
    return 100 * (np.exp(y) - 1)


for var in ["Treated", "Tmean"]:
    print(var)
    for project in pu.PROJECTS:
        mod = base_cooling_models[project]
        print(f"{project}: {y_transform(mod.params[var]):.2f}%")
    print()


def y_transform2(y):
    return (1 + (np.exp(y) - 1)) ** 10


print("Tmean impact for 10$\degree$F")
for project in pu.PROJECTS:
    mod = base_cooling_models[project]
    print(f"{project}: {y_transform2(mod.params[var]):.2f}")

# Supplemental figures and tables

## Figures S1 and S2 - Cooling load plots with different units

In [ ]:
for cooling_load_unit in ["ktonhr", "kWhe_per_m2"]:
    data = {project: pu.load(project) for project in pu.PROJECTS}
    for project in pu.PROJECTS:
        if cooling_load_unit == "ktonhr":  # floorspace is in 1e3 m2
            data[project].cooling = (
                data[project].cooling * pu.floorspace[project] / pu.tonhr_to_MJ
            )
            fig_name = pu.si_path / "Figure S1.pdf"
        elif cooling_load_unit == "kWhe_per_m2":
            data[project].cooling = data[project].cooling * pu.cop_to_kWe_per_MJc(5.5)
            fig_name = pu.si_path / "Figure S2.pdf"
    make_plot(data, cooling_load_unit, fig_name)

## Table S2

In [ ]:
with open(pu.si_path / "Table S2.tex", "w") as fw:
    fw.write(
        pu.si_table(
            base_cooling_models,
            pu.PROJECTS,
            caption="Additional numerical data for the models shown in Figure 2 (cooling).",
            label="table:s2",
        )
    )

## Figure S8 Linear model for cooling load

In [ ]:
## data = {project: pu.load(project) for project in pu.PROJECTS}

cooling_load_unit = "MJ_per_m2"
data = {project: pu.load(project) for project in pu.PROJECTS}

base_cooling_models_linear = {}

f, ax = plt.subplots(2, 3, figsize=(pu.PAGE_WIDTH, pu.ROW_HEIGHT * 2), sharex=True)
ax = ax.flatten()
for i, project in enumerate(pu.PROJECTS):
    weekend = pu.choose_weekend[project]
    pu.plot_base(
        data[project],
        fax=(f, ax[i]),
        weekend=weekend,
        s=2,
        cooling_load_unit=cooling_load_unit,
    )
    base_cooling_models_linear[project] = pu.create_model(
        data[project], fax=(f, ax[i]), weekend=weekend, log=False
    )
    _ = pu.create_model(
        data[project], fax=(f, ax[i]), weekend=weekend, ls=[(0, (1, 1)), "-."]
    )
    ax[i].set_title(f"{pu.plot_numbers[i]}) {project}")
for a in ax:
    a.xaxis.set_tick_params(labelbottom=True)
f.tight_layout()
pu.add_legend((f, ax))

if pu.save_fig:
    f.savefig(pu.si_path / f"Figure S8 A-F.pdf")

In [ ]:
f, ax = plt.subplots(1, 2, figsize=(pu.PAGE_WIDTH, 0.75 * pu.ROW_HEIGHT))
ax = ax.flatten()
for i, project in enumerate(pu.PROJECTS):
    print(project)
    pu.ci_plot(
        ax[0],
        i,
        base_cooling_models_linear[project],
        "Treated",
        color=pu.dark_grey,
        verb=True,
        log=False,
    )
    pu.ci_plot(
        ax[1],
        i,
        base_cooling_models_linear[project],
        "Tmean",
        color=pu.dark_grey,
        verb=True,
        log=False,
    )
    print()

for a in ax:
    a.set_xticks(range(len(pu.PROJECTS)))
    a.set_xticklabels(pu.PROJECTS, rotation=25, ha="right")
ax[0].set_ylabel("Impact (MJ/m$^2$/day)")
ax[1].set_ylabel("Impact (MJ/m$^2$/day/$\degree$F)")
ax[0].set_ylim(top=0)
ax[1].set_ylim(bottom=0)

title0 = (
    f"{pu.plot_numbers[6]}) "
    + "Impact of 2$\degree$F set point increase on energy for cooling"
)
title1 = f"{pu.plot_numbers[7]}) " + "Impact of mean daily OAT on energy for cooling"
ax[0].set_title(title0, ha="left", loc="left")
ax[1].set_title(title1, ha="left", loc="left")

f.tight_layout(w_pad=2.0)
plt.subplots_adjust(top=0.85, bottom=0.27)
if pu.save_fig:
    f.savefig(pu.si_path / "Figure S8 G-H.pdf")

## Table S6

In [ ]:
with open(pu.si_path / "Table S6.tex", "w") as fw:
    fw.write(
        pu.si_table(
            base_cooling_models_linear,
            pu.PROJECTS,
            caption="Additional numerical data for models shown in Figure S8 (alternate linear model for cooling).",
            label="table:s6",
        )
    )